### Symantec - Data Analysis Exercise
##### Submiited by: Aditya Shirode

After reading about the data at hand, which is in JSON Lines format, the initial thoughts were -  
- Do I need to load the data into a database?
- If yes, what would be the best way to do that?
- If not, then do I need to store it in some data structure locally, or can I just analyze on the fly?

Both of the options mentioned above are possible, but to make the experiment flexible and extensible, it was pertinent that we store the data in some database.  

Since the data is in a flexible format (JSON), we need a database which is not constrained by a rigid, pre-defined structure, hence NoSQL.  
Among NoSQL databases, MongoDB - a document oriented database, which can easily work with JSON format, seems perfect fit.

##### What is [MongoDB](https://www.mongodb.com/what-is-mongodb):  
A NoSQL, document database  
Free and open source  

What we gain from MongoDB:
- Native data representation is JSON
- Can do simple real time aggregation, which covers the basic queries we need

We will use PyMongo to load the data and answer our queries  

###### [PyMongo](http://api.mongodb.com/python/current/index.htm):  
- PyMongo is a driver for MongoDB in Python  
- It connects Python application to *'mongod'* (MongoDB Daemon) using BSON (Binary JSON)  

## 1. Setting up MongoDB

##### How to setup MongoDB:  
- Download the MongoDB installer for the community release [here](https://www.mongodb.com/download-center?jmp=nav#community)
- Select the version based on the operating system you are running this application on
- Run the installer after download is completed
- Go through the installation with default settings


- Navigate to the MongoDB installation on your local machine (For Windows, the default is 'C:\Program Files\MongoDB\Server\3.4')
- Create two new folders inside the above folder: */data* and */log*
- Navigate to the */data/* folder and create another folder */db*
- Now, the directory structure inside the MongoDB folder should be:
    - */bin*
    - */data/db*
    - */log*
    - and some other files

- Now open a terminal and navigate to the */bin* folder mentioned above
- Run the following command:  
> ```mongod --directoryperdb --dbpath 'C:\\Program Files\\MongoDB\\Server\\3.4\\data\\db'   
--logpath 'C:\\Program Files\\MongoDB\\Server\\3.4\\log\\mongo.log'  
--logappend --rest --install```

- This command will start the MongoDB daemon with settings as:
    - **--directoryperdb**: Uses a separate directory to store data for each database (separation per database)
    - **--dbpath [path]**: The directory where the mongod instance stores its data (In our case ..\data\db)
    - **--logpath [path]**: Sends all diagnostic logging information to a log file (We store our log in ..\log\Mongo.log)
    - **--logappend**: Append new entries to the end of the existing log file
    - **--rest**: Enables the simple REST API, to use MongoDB as a service
    - **--install**: Installs the service
    
- Start the service by running following command:  
    > ```net start MongoDB```
    
Note: All of this is explained in detail on [MongoDB Windows Installation](https://docs.mongodb.com/manual/tutorial/install-mongodb-on-windows/) and [MongoDB Documentation](https://docs.mongodb.com/manual/reference/program/mongod/)  

###### Installing PyMongo:
Run following command to [install](http://api.mongodb.com/python/current/installation.html) PyMongo:  
> ```pip install pymongo```

Tuts:  
https://www.youtube.com/watch?v=GSL8JpyAjsE  
https://www.youtube.com/watch?v=pWbMrx5rVBE  
http://api.mongodb.com/python/current/tutorial.html  

----------

## 2. Loading the Data

Now that we have a running instance of MongoDB, let's load our data in it using *PyMongo*.

In [1]:
# Setup connection to client
from pymongo import MongoClient
client = MongoClient()

In [2]:
# Create a Database
db = client.symantec

In [3]:
# Create a collection - group of documents
collection = db.people

In [4]:
import os
import json

# Directory where our 'JSONL' file is stored
DATA_DIR = os.path.join(os.getcwd(), 'data')
filename = 'ida_wrangling_exercise_data.2017-02-13.jsonl'

# Open the file, parse entry one by one, and load into database
with open(os.path.join(DATA_DIR, filename)) as f:
    for i, line in enumerate(f):
        line = line.strip()
        collection.insert_one(json.loads(line))

In [5]:
# Check if the data is loaded by randomly checking one entry
from pprint import pprint
pprint(collection.find_one())

{'_id': ObjectId('59c376c1b1531e28986dd31f'),
 'address': {'city': 'Hoodburgh',
             'state': 'RI',
             'street': '86314 David Pass Apt. 211',
             'zip': '83973'},
 'dob': '1971-06-30',
 'email': 'opark@hotmail.com',
 'id': '01d68a4c598a45559c06f4df0b3d82cb',
 'name': {'firstname': 'Cynthia', 'lastname': 'Dawson', 'middlename': 'Claire'},
 'phone': '624-869-4610',
 'record_date': '2006-07-08T09:02:13',
 'ssn': 'xxx-xx-2412'}


In [6]:
# Total no of records
n = collection.count()
n

150000

-------------------------

## 3. Analysis

Now that we have our data loaded, we can find answers to our questions.

##### Q1
Start by making a list of all of the nested named fields that appear in any record.  
Concatenate nested field names using a period '.' to defind named fields for nested records.  
Present the list in alphabetical order.

In [7]:
# Maintain a set of keys found
keys = set()

# A function similar to flattening a list of lists
def flatten(record, parent=list()):
    ''' Function to flatten a record '''
    for key, value in record.items():
        if type(value) is dict: # If nested record, recurse with updated parent
            flatten(value, parent + [key])
        if parent:
            keys.add('.'.join(parent + [key]))
        else:
            keys.add(key)
                
for record in collection.find():
    flatten(record)

# Remove the '_id' field generated by MongoDB while inserting the records
keys.remove('_id')

print(sorted(keys))

['address', 'address.city', 'address.state', 'address.street', 'address.zip', 'dob', 'email', 'id', 'name', 'name.firstname', 'name.lastname', 'name.middlename', 'phone', 'record_date', 'ssn']


It would have been better to keep track of these when inserting records into the database,  
because we are iterating over every record again to calculate the desired result.

There are complex way to do the same recursion, by utilizing MongoDB functions such as 'MapReduce', 'aggregate'; but given the short amount of time, let's go the straightforward way.  
Given more time, it'd be interesting to see which method performs fastest.

##### Q2
- What percentage of the records contain the field?
- What are the five most common values of the field?

Updating the above query a bit, we can update a counter and find out these answers.  

We can utilize the the aggregate function that MongoDB provides, but we'll lose the hierarchy in doing that.  
We can write a helper Script function outside, but that would not be completely Pythonic.

In [8]:
# Maintain a set of keys found
keys_counter = dict()

# A function similar to flattening a list of lists
def flatten(record, parent=list()):
    ''' Function to flatten a record '''
    for key, value in record.items():
        if type(value) is dict: # If nested record, recurse with updated parent
            flatten(value, parent + [key])
        
        # Instead of adding the key found to a set, we are maintaing a dictionary counter
        key = '.'.join(parent + [key])
        keys_counter[key] = keys_counter.get(key, 0) + 1
                
for record in collection.find():
    flatten(record)

# Remove the '_id' field generated by MongoDB while inserting the records
_ = keys_counter.pop('_id')
# assert _ == n # The no of '_id' popped out should be equal to the total number of records

In [9]:
pprint(keys_counter)

{'address': 136426,
 'address.city': 61233,
 'address.state': 61233,
 'address.street': 61233,
 'address.zip': 61233,
 'dob': 143874,
 'email': 130880,
 'id': 150000,
 'name': 148062,
 'name.firstname': 105004,
 'name.lastname': 105004,
 'name.middlename': 43669,
 'phone': 140261,
 'record_date': 150000,
 'ssn': 142444}


In [10]:
# OrderedDict to maintain the alphabetical sorted order
from collections import OrderedDict
keys_counter_ordered = OrderedDict(sorted(keys_counter.items()))

# Since we have count corresponding to each key, 
# And we have n = total no of records,
# We can calculate percentage easily
for key, val in keys_counter_ordered.items():
    print("{:16} {:6.2f}%".format(key, val * 100 / n))

address           90.95%
address.city      40.82%
address.state     40.82%
address.street    40.82%
address.zip       40.82%
dob               95.92%
email             87.25%
id               100.00%
name              98.71%
name.firstname    70.00%
name.lastname     70.00%
name.middlename   29.11%
phone             93.51%
record_date      100.00%
ssn               94.96%


In [11]:
# Implementing Counter to quickly extract top 5 keys
# Can use a custom data structure (heap) for it if needed

from collections import Counter
keys_counter = Counter(keys_counter)

In [12]:
# Five most common keys
keys_counter.most_common(5)

[('id', 150000),
 ('record_date', 150000),
 ('name', 148062),
 ('dob', 143874),
 ('ssn', 142444)]

##### Q3
How many distinct first names appear in this data set?

We have two hierarchies for first name: name and name.firstname  
Let's count both separately and then find the union.

In [13]:
# Using MongoDB's distinct() function
distinct_firstnames = set(collection.distinct("name.firstname"))

*collection.distinct("name")* returns every unique entry in "name" field; that includes the nested {firstname, lastname} fields as well.  

Iterating over those, we ignore them, and parse the other names by splitting at whitespace and considering the first word to be the first name, we make additions to above set.

In [14]:
# For every distinct "name" in database
for nm in collection.distinct("name"):
    # If that name is not a nested record
    if type(nm) is not dict:
        # Split the name at space and consider first word to be first name
        fname, *lname = nm.split(" ")
        distinct_firstnames.add(fname)

In [15]:
len(distinct_firstnames)

695

##### Q4
How many distinct street names appear in this data set?

Similar to first names, we again have two fields which represent street name -  
One is straightforward: *"address.street"*  
and the other is *"address"* itself

We need to extract the street name from the string address and maintain a set of unique ones

In [16]:
distinct_streets = set()

# Ignore the digits at the start
# After you encounter the first space, 
# try to find two groups of alphabets (lower/upper case), seperated by a space
street_regex = r'(?:\d+).*?\s+([A-z]+\s[A-z]+)'

By modifying this regular expression, we can refine our criterion for "street names".  
Right now, it captures two-worded street names that follow a block number. [(Reference)](https://en.wikipedia.org/wiki/Street_or_road_name)

In [17]:
import re

# For every distinct "address"
for addr in collection.distinct("address"):    
    # If the address is nested, just consider the "street"
    if type(addr) is dict:
        addr = addr.get("street", "")
    m = re.match(street_regex, addr)
    if m:
        distinct_streets.add(m.group(1))

In [18]:
len(distinct_streets)

79999

You can implement a regex in MongoDB's find() function; but it will return you matching records, not individual fields  
You have to run a regex again on the result, which is double work. Hence, the above approach.

In [19]:
collection.find({
    "address": {"$regex": r'(?:\d+).*?\s+(.+)'}
}).count()

72485

##### Q5
What are the 5 most common US area codes in the phone number field?

In [20]:
area_codes = {}
phone_regex = r'(?:\+?)\(?([1-9]\d{2})\)?'

Again, we take a similar approach as streets, by defining a 'regular expression' to extract the 3-digit area code from the phone number field.  
We store it in a dictionary, which acts as a counter.

In [21]:
# For every record in database
for record in collection.find():
    # If it has a "phone" field
    phone = record.get("phone", None)
    if phone:
        # Try to match it to the regex to extract area code
        m = re.match(phone_regex, phone)
        if m:
            code = m.group(1)
            # Increment the count for that area code
            area_codes[code] = area_codes.get(code, 0) + 1

In [22]:
Counter(area_codes).most_common(5)

[('913', 119), ('947', 117), ('488', 117), ('796', 114), ('270', 113)]

It seems 'Kansas' is the most widely observed area code in our database, followed closely by 'Michigan' and the 'mysterious' 488.

In [23]:
# Clean-up
db.people.drop()

-------------------------------------------

## Answers

#### Q1: List of all of the nested named fields
['address', 'address.city', 'address.state', 'address.street', 'address.zip', 'dob', 'email', 'id', 'name', 'name.firstname', 'name.lastname', 'name.middlename', 'phone', 'record_date', 'ssn']

#### Q2a: What percentage of the records contain the field?
| Field | Percentage |
| :------- | -------: |
|address          | 90.95% |
|address.city     | 40.82% |
|address.state    | 40.82% |
|address.street   | 40.82% |
|address.zip      | 40.82% |
|dob              | 95.92% |
|email            | 87.25% |
|id              | 100.00% |
|name             | 98.71% |
|name.firstname   | 70.01% |
|name.lastname    | 70.01% |
|name.middlename  | 29.11% |
|phone            | 93.51% |
|record_date     | 100.00% |
|ssn              | 94.96% |

#### Q2b: What are the five most common values of the field?
| Field | Frequency |
| :---- | --------: |
| 'id' | 150014 |
| 'record_date' | 150014 |
| 'name' | 148076 |
| 'dob' | 143888 |
| 'ssn' | 142458 |
 
#### Q3: How many distinct first names appear in this data set?
695

#### Q4: How many distinct street names appear in this data set?
79999

#### Q5: What are the 5 most common US area codes in the phone number field?
| Area Code | Frequency |
| :-------: | :-------: |
| (913) | 119 |
| (947) | 117 |
| (488) | 117 |
| (796) | 114 |
| (270) | 113 |

--------------------------------

#### Conclustion

Overall, storing our records in a NoSQL (MongoDB) database, didn't help us much.  
It made easy to access and query the database, thanks to PyMongo wrapper; but we didn't utilize the underlying efficiency of MongoDB.  
It was considerably easy to load the JSONL formatted data into MongoDB.

As for analysis, we mostly parsed all the records for each of our questions.  
To achieve efficiency and avoid repeat querying for records, we can iterate over each record once (maybe even while loading it into database), and calculate the required statistics at that time.  
But, with current situation, we were able to answer all of our queries very quickly.  
Depending on what more are we supposed to do with the data, and based on the type and frequency of queries, we can modify the functions and use some Hadoop speedups or non-structured querying advantages.